In [7]:
from gensim.models import Word2Vec
from sequence.sequence_manager import *
from sequence.sequence_plotter import *

In [2]:
path = "/home/chris/Scrivania/url2vec/dataset/depth-100k/seqLen-10/cs.illinois.eduRandomWalkLists.depth.100000.seqLen.10/"
rwwl_map_path = path + "sequencesMapUrl.txt"
rwwl_seq_path = path + "sequencesIDs.txt"

In [8]:
vocab_sequences = get_seq(rwwl_seq_path, 1)
train_sequences = get_seq(rwwl_seq_path, 1)

In [6]:
model = Word2Vec(min_count=1)
model.build_vocab(vocab_sequences)
model.train(train_sequences)

890323

In [26]:
seq_map = get_seq_map(rwwl_map_path)
url_key = []
for i in seq_map:
    print model[i]

[ 0.20388715  0.21801275 -0.49791577  0.06093926  0.00633004  0.16863158
  0.17820698 -0.1745581   0.12666918  0.28151938 -0.19326845  0.01006678
 -0.20614941 -0.030797   -0.04466429 -0.27982599  0.14226398 -0.06431222
  0.29759553 -0.41963655 -0.09063941 -0.22075374 -0.02462082 -0.19624837
  0.31406793  0.06346641 -0.12578724  0.15149765  0.01253543  0.15435898
 -0.25632367  0.27785951  0.01065231  0.39805838  0.03456411  0.23165721
 -0.34637159  0.21652341  0.11107881  0.08652655  0.34225047  0.03350877
 -0.02771597  0.43714458 -0.14885812  0.17337239  0.06295291  0.14748862
  0.01147121  0.19741416  0.40998265  0.06972668  0.06517275 -0.07810906
 -0.24728744  0.03426838  0.02089745  0.00771588  0.02678175 -0.16643278
  0.07595026  0.00977347  0.29381198 -0.12374856  0.10439787  0.32962522
  0.17554961  0.11128805 -0.36220175  0.30742377 -0.13894179 -0.07827676
 -0.10703    -0.09320759  0.27695403 -0.23913831 -0.13463245 -0.21945514
  0.20376484  0.06396154 -0.15398341 -0.18782794  0